[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/Word2vec.ipynb)

# Word2vec

In [1]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

In [1]:
# Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [2]:
# 因子数
factors = 100
# エポック数
n_epochs = 30
# windowサイズ
window = 100
# スキップグラム
use_skip_gram = 1
# 階層的ソフトマックス
use_hierarchial_softmax = 0
# 使用する単語の出現回数のしきい値
min_count = 5

In [4]:
movie_content = movies.copy()
# tagが付与されていない映画もあるが、genreはすべての映画に付与されている
# tagとgenreを結合したものを映画のコンテンツ情報として似ている映画を探して推薦していく
# tagがない映画に関しては、NaNになっているので、空のリストに変換してから処理をする
movie_content["tag_genre"] = movie_content["tag"].fillna("").apply(list) + movie_content["genre"].apply(list)
movie_content["tag_genre"] = movie_content["tag_genre"].apply(lambda x: set(map(str, x)))

In [ ]:
!pip install gensim==4.0.1

In [6]:
import gensim

# タグとジャンルデータを使って、word2vecを学習する
tag_genre_data = movie_content.tag_genre.tolist()
model = gensim.models.word2vec.Word2Vec(
    tag_genre_data,
    vector_size=factors,
    window=window,
    sg=use_skip_gram,
    hs=use_hierarchial_softmax,
    epochs=n_epochs,
    min_count=min_count,
)


/Users/mk/.pyenv/versions/3.7.8/envs/recsys-test3.7.8/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [7]:
# animeタグに似ているタグを確認
model.wv.most_similar('anime')

[('studio ghibli', 0.8167880177497864),
 ('zibri studio', 0.8076589107513428),
 ('pelicula anime', 0.7783956527709961),
 ('hayao miyazaki', 0.7779476046562195),
 ('miyazaki', 0.754835307598114),
 ('japan', 0.6330965161323547),
 ('Animation', 0.5467621088027954),
 ('girl', 0.528790295124054),
 ('wilderness', 0.5270888805389404),
 ('dragon', 0.5217347741127014)]

In [8]:
# 各映画のベクトルを計算する
# 各映画に付与されているタグ・ジャンルのベクトルの平均を映画のベクトルとする
movie_vectors = []
tag_genre_in_model = set(model.wv.key_to_index.keys())

titles = []
ids = []

for i, tag_genre in enumerate(tag_genre_data):
    # word2vecのモデルで使用可能なタグ・ジャンルに絞る
    input_tag_genre = set(tag_genre) & tag_genre_in_model
    if len(input_tag_genre) == 0:
        # word2vecに基づいてベクトル計算できない映画にはランダムのベクトルを付与
        vector = np.random.randn(model.vector_size)
    else:
        vector = model.wv[input_tag_genre].mean(axis=0)
    titles.append(movie_content.iloc[i]["title"])
    ids.append(movie_content.iloc[i]["movie_id"])
    movie_vectors.append(vector)


In [10]:
import numpy as np

# 後続の類似度計算がしやすいように、numpyの配列で保持しておく
movie_vectors = np.array(movie_vectors)

# 正規化したベクトル
sum_vec = np.sqrt(np.sum(movie_vectors ** 2, axis=1))
movie_norm_vectors = movie_vectors / sum_vec.reshape((-1, 1))


In [11]:
# まだ評価していない映画で、類似度が高いアイテムを返す関数
def find_similar_items(vec, evaluated_movie_ids, topn=10):
    score_vec = np.dot(movie_norm_vectors, vec)
    similar_indexes = np.argsort(-score_vec)
    similar_items = []
    for similar_index in similar_indexes:
        if ids[similar_index] not in evaluated_movie_ids:
            similar_items.append(ids[similar_index])
        if len(similar_items) == topn:
            break
    return similar_items

In [13]:
# ユーザーが評価値４以上をつけた映画
movielens_train_high_rating = movielens_train[movielens_train.rating >= 4]
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()

id2index = dict(zip(ids, range(len(ids))))
pred_user2items = dict()
for user_id, data in movielens_train_high_rating.groupby("user_id"):
    evaluated_movie_ids = user_evaluated_movies[user_id]
    movie_ids = data.sort_values("timestamp")["movie_id"].tolist()[-5:]

    movie_indexes = [id2index[id] for id in movie_ids]
    user_vector = movie_norm_vectors[movie_indexes].mean(axis=0)
    recommended_items = find_similar_items(user_vector, evaluated_movie_ids, topn=10)
    pred_user2items[user_id] = recommended_items
pred_user2items

{1: [4306, 2054, 7369, 5452, 2080, 6559, 36708, 5500, 44022, 2804],
 2: [1198, 1196, 1097, 3578, 1204, 7153, 2028, 527, 3471, 940],
 3: [27797, 3874, 61071, 26989, 3194, 5499, 5524, 51314, 4964, 8503],
 4: [2125, 1073, 551, 2797, 33836, 374, 1028, 4184, 2054, 2953],
 5: [1193, 3168, 1276, 928, 3504, 1161, 3741, 2858, 3201, 2729],
 6: [5445, 198, 2594, 5881, 292, 1301, 1240, 541, 3156, 8914],
 7: [2935, 955, 922, 7080, 4357, 946, 3307, 4914, 947, 910],
 8: [33437, 4956, 2764, 517, 36529, 7386, 2871, 2912, 58025, 6297],
 9: [2890, 1953, 1162, 3168, 319, 665, 1213, 4399, 33166, 1884],
 10: [1950, 3504, 1953, 920, 1925, 2396, 909, 1961, 1721, 1927],
 11: [1784, 3259, 339, 1393, 11, 1307, 8969, 7, 3893, 357],
 12: [1438, 6385, 1805, 11, 517, 1876, 2594, 3418, 1674, 4699],
 13: [1784, 1084, 909, 356, 1213, 3201, 1276, 1263, 1230, 3100],
 14: [3988, 8516, 586, 2161, 3489, 2125, 2804, 2054, 7046, 1265],
 16: [748, 2288, 1301, 5522, 5046, 2662, 3699, 5881, 2528, 674],
 17: [1253, 1254, 2529, 10

In [14]:
# user_id=2のユーザーが学習データで、4以上の評価を付けた映画一覧
movielens_train_high_rating[movielens_train_high_rating.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
8381,2,1210,4.0,868245644,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Sci-Fi]","[desert, fantasy, sci-fi, space, lucas, gfei o...",10.0


In [15]:
# user_id=2に対するおすすめ(1198, 1196, 1097)
movies[movies.movie_id.isin([1198, 1196, 1097])]

,movie_id,title,genre,tag
1075,1097,E.T. the Extra-Terrestrial (1982),"[Children, Drama, Sci-Fi]","[speilberg, steven spielberg, spielberg/lucas,..."
1171,1196,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Sci-Fi]","[lucas, george lucas, george lucas, gfei own i..."
1173,1198,Raiders of the Lost Ark (Indiana Jones and the...,"[Action, Adventure]","[egypt, lucas, seen more than once, dvd collec..."
